# Controller #

Import all neccessary packages

In [ ]:
import os
import pandas as pd
import numpy as np

from drivers.sandbox import Sandbox
from drivers.generator import Generator
from drivers.optimiser import Optimiser

from inference_toolbox.domain import Domain
from inference_toolbox.parameter import Parameter
from inference_toolbox.model import Model
from inference_toolbox.likelihood import Likelihood

current_directory = os.getcwd()
if current_directory != '/project/':
    os.chdir('/project/')

Define how the data is to be generated and what default inputs should be used

In [2]:
data_params = {
    'data_type': 'gridded',
    'output_header': 'Concentration',
    'log':True,
    'grid_size': [200,200,50],
    'target': False,
    'data_path':'data'
}

default_params = {
    'infered_params':pd.Series({
        'model_params':pd.Series({
            'I_y': Parameter('I_y', prior_select = 'gamma', default_value=0.1).add_prior_param('mu', 0.1).add_prior_param('sigma',0.1),
            'I_z': Parameter('I_z', prior_select = 'gamma', default_value=0.1).add_prior_param('mu', 0.1).add_prior_param('sigma',0.1),
            'Q': Parameter('Q', prior_select = 'gamma', default_value=3e13).add_prior_param('mu', 3e13).add_prior_param('sigma',1e13),
        }),
        'likelihood_params':pd.Series({})
    }),
    'model':Model('log_gpm_alt_norm').add_model_param('H',10),
    'likelihood': Likelihood('gaussian_fixed_sigma').add_likelihood_param('sigma',1),
    'sampler': {
        'n_samples': 10000,
        'n_chains': 3,
        'thinning_rate': 1
    }
}

## Sandbox ##

Create an instance with the inputted default parameters and visualise it in different ways

In [3]:
sandbox = Sandbox(results_name='gridded_data_200_200_50_real_fixed_sigma', 
                  default_params=default_params, 
                  data_params=data_params)

visualiser = sandbox.run()
visualiser.get_summary()
visualiser.get_traceplot()
visualiser.get_autocorrelations()

domain = Domain('cone_from_source_z_limited', resolution=80)
domain.add_domain_param('r', 100)
domain.add_domain_param('theta', np.pi/8)
domain.add_domain_param('source', [0,0,10])

visualiser.visualise_results(domain = domain, name = 'small_scale_3D_plots', title='Log Concentration of Droplets', log_results=False)
visualiser.animate(name = 'small_scale_3D_plots')

Loading Chain Samples...
Loading Samples...
Traceplot 1 already exists
Traceplot 2 already exists
Traceplot 3 already exists
Autocorrelations plot 1 already exists
Autocorrelations plot 2 already exists
Autocorrelations plot 3 already exists
Plots already exist!
Animation already exist!


## Generator ##

Generate different instances by varying the hyperparameters with respect to their defaults

In [4]:
generator = Generator(results_name='gridded_data_200_200_50_real_fixed_sigma', 
                  default_params=default_params, 
                  data_params=data_params)

Vary one hyperparameter

In [5]:
analysis_iterations = {
    'parameters':
    [
        'likelihood_sigma', 
    ],
    'values':
    [
        np.array([0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2, 2.2, 2.4, 2.6, 2.8, 3, 3.2, 3.4, 3.6, 3.8, 4])
    ]
}

for i in range(len(analysis_iterations['parameters'])):
    parameter = analysis_iterations['parameters'][i]
    print('Working on varying ' + parameter + '...')
    values = analysis_iterations['values'][i]
    inputs = generator.vary_one_parameter(parameter, values, xscale='linear', plot=True)

Working on varying likelihood_sigma...


Vary two hyperparameters

In [6]:
analysis_iterations = {
    'parameters_1':
    [
        'I_y_sigma', 
    ],
    'parameters_2':
    [
        'likelihood_sigma', 
    ],
    'values_1':
    [
        np.array([1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]),
    ],
    'values_2':
    [
        np.array([0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2]),
    ]
}

for i in range(len(analysis_iterations['parameters_1'])):
    parameter_1 = analysis_iterations['parameters_1'][i]
    parameter_2 = analysis_iterations['parameters_2'][i]

    print('Working on varying ' + parameter_1 + '...')
    values_1 = analysis_iterations['values_1'][i]
    values_2 = analysis_iterations['values_2'][i]
    inputs = generator.vary_two_parameters(parameter_1, parameter_2, values_1, values_2, scale_1='log', scale_2='linear', plot=True)

Working on varying I_y_sigma...
Generating instance 1...


sample: 100%|██████████| 12500/12500 [00:10<00:00, 1182.80it/s, 511 steps of size 2.65e-07. acc. prob=0.81] 


Generating instance 2...


sample: 100%|██████████| 12500/12500 [00:01<00:00, 6498.00it/s, 3 steps of size 7.20e-01. acc. prob=0.89]


Generating instance 3...


...

sample: 100%|██████████| 12500/12500 [00:01<00:00, 6273.87it/s, 3 steps of size 6.97e-01. acc. prob=0.93]


Generating instance 69...


sample: 100%|██████████| 12500/12500 [00:02<00:00, 6149.81it/s, 3 steps of size 7.22e-01. acc. prob=0.92]


Generating instance 70...


sample: 100%|██████████| 12500/12500 [00:02<00:00, 6211.07it/s, 3 steps of size 7.38e-01. acc. prob=0.92]


## Optimiser ##

Optimise the default construction using Bayesian Optimisation

In [ ]:
optimising_parameters = {
                    'I_y_mu': [1e-2, 10],
                    'I_y_sigma': [1e-2, 10],
                    'I_z_mu': [1e-2, 10],
                    'I_z_sigma': [1e-2, 10],
                    'Q_mu': [1e9, 1e18],
                    'Q_sigma': [1e9, 1e18],
                }

optimiser = Optimiser(results_name='gridded_data_200_200_50_real_fixed_sigma', 
                  default_params=default_params, 
                  data_params=data_params)

study = optimiser.run(n_trials=10, optimiser_name='optimiser_1', optimising_parameters=optimising_parameters, index_name='aic')
optimiser.get_plots(study)

optimiser.run_best_params(study)